In [1]:
import pandas as pd

from sklearn.svm import SVR #SVR
from sklearn.tree import DecisionTreeRegressor #decisiontreeregressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split as holdout

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#import library

from google.cloud import bigquery

client = bigquery.Client()
query = """
SELECT 
a.* except (loan_adm_pemerintahan_pertahanan_jaminan_sosial_wajib_112023,
simpanan_giro_nominal_112023,
simpanan_berjangka_nominal_112023,
simpanan_tabungan_nominal_112023,
loan_flat_dan_apartemen_112023,
loan_industri_pengolahan_112023,
loan_informasi_komunikasi_112023,
loan_jasa_lainnya_112023,
loan_jasa_pendidikan_112023,
loan_jasa_perusahaan_112023,
loan_kendaraan_bermotor_112023,
loan_kesehatan_dan_kegiatan_lain_112023,
loan_keuangan_dan_asuransi_112023,
loan_konstruksi_112023,
loan_lainnya_112023,
loan_pengadaan_air_pengelolaan_sampah_limbah_daur_ulang_112023,
loan_pengadaan_listrik_dan_gas_112023,
loan_penyedia_akm_makan_minum_112023,
loan_perdagangan_besar_ecer_reparasi_mobil_motor_112023,
loan_pertambangan_penggalian_112023,
loan_pertanian_kehutanan_perikanan_112023,
loan_real_estate_112023,
loan_ruko_rukan_112023,
loan_rumah_tinggal_112023,
loan_transportasi_pergudangan_112023),
b.jenis_kpi,
b.jenis_simpanan,
b.real_2018,
b.real_2019,
b.real_2020,
b.real_2021,
b.real_2022,
b.real_2023,
b.pred_2024,
b.pred_2025,
b.pred_2026,
b.pred_2027,
b.pred_2028
FROM 
`ai4indonesia.geoplan_sbx.ai4_reff_kab`  as a inner join
`ai4indonesia.geoplan_sbx.ai4_data_simpanan_kab`  as b
on a.bps_kab = b.kab_id
"""
query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()

df.head()

,kab,bps_kab,prov,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,...,real_2019,real_2020,real_2021,real_2022,real_2023,pred_2024,pred_2025,pred_2026,pred_2027,pred_2028
0,KOTA PONTIANAK,6171,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,...,14176141.56,15636562.28,17466906.94,19076608.21,19685780.82,21337157.77,23183685.09,24793584.13,25402713.87,27054071.57
1,KOTA PONTIANAK,6171,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,...,11441612.99,12359011.23,11537865.50,12355606.65,13573471.32,14819544.15,14134925.41,14952686.10,16170468.36,17416586.17
2,KOTA PONTIANAK,6171,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,...,4346386.21,4511502.50,5689567.99,6322022.97,6908151.03,6934861.54,8093113.49,8725520.38,9311645.83,9338450.66
3,KOTA SINGKAWANG,6172,KALIMANTAN BARAT,242146,74267,470.199797,452.0,514.985335,21379,20683,...,2940837.55,3441112.76,3814378.76,3926997.56,3887159.25,4540767.94,4914045.53,4949971.63,5063504.80,5640435.20
4,KOTA SINGKAWANG,6172,KALIMANTAN BARAT,242146,74267,470.199797,452.0,514.985335,21379,20683,...,1551944.28,1498317.21,1341095.67,1431449.20,1432910.42,1470636.39,1313414.30,1358092.08,1450905.05,1442955.19


In [5]:
df = df.set_index('bps_kab')
df.fillna(0, inplace=True)

df.head()

,kab,prov,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,...,real_2019,real_2020,real_2021,real_2022,real_2023,pred_2024,pred_2025,pred_2026,pred_2027,pred_2028
bps_kab,,,,,,,,,,,,,,,,,,,,,
6171,KOTA PONTIANAK,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,52263,...,14176141.56,15636562.28,17466906.94,19076608.21,19685780.82,21337157.77,23183685.09,24793584.13,25402713.87,27054071.57
6171,KOTA PONTIANAK,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,52263,...,11441612.99,12359011.23,11537865.50,12355606.65,13573471.32,14819544.15,14134925.41,14952686.10,16170468.36,17416586.17
6171,KOTA PONTIANAK,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,52263,...,4346386.21,4511502.50,5689567.99,6322022.97,6908151.03,6934861.54,8093113.49,8725520.38,9311645.83,9338450.66
6172,KOTA SINGKAWANG,KALIMANTAN BARAT,242146,74267,470.199797,452.0,514.985335,21379,20683,20221,...,2940837.55,3441112.76,3814378.76,3926997.56,3887159.25,4540767.94,4914045.53,4949971.63,5063504.80,5640435.20
6172,KOTA SINGKAWANG,KALIMANTAN BARAT,242146,74267,470.199797,452.0,514.985335,21379,20683,20221,...,1551944.28,1498317.21,1341095.67,1431449.20,1432910.42,1470636.39,1313414.30,1358092.08,1450905.05,1442955.19


Index(['kab', 'prov', 'total_pnddk', 'jmlh_kk', 'luas_wilayah_gis',
       'luas_wil_dukcapil', 'pnddk_per_m', 'usia_20sd24', 'usia_25sd29',
       'usia_30sd34', 'usia_35sd39', 'usia_40sd44', 'lulus_sma', 'd1_d2', 'd3',
       's1', 's2', 's3', 'tdk_bekerja', 'asn', 'petani', 'peternak', 'nelayan',
       'pljr_mhswa', 'pedagang', 'perawat', 'popses_a1', 'popses_a2',
       'popses_b', 'popses_c1', 'popses_c2', 'popses_d', 'popses_e',
       'jmlh_kk_a1', 'jmlh_kk_a2', 'jmlh_kk_b', 'jmlh_kk_c1', 'jmlh_kk_c2',
       'jmlh_kk_d', 'jmlh_kk_e', 'mostses', 'avrg_income', 'jenis_kpi',
       'jenis_simpanan', 'real_2018', 'real_2019', 'real_2020', 'real_2021',
       'real_2022', 'real_2023', 'pred_2024', 'pred_2025', 'pred_2026',
       'pred_2027', 'pred_2028'],
      dtype='object')

In [6]:
target = [
'real_2018',
'real_2019',
'real_2020',
'real_2021',
'real_2022',
'real_2023',
'pred_2024',
'pred_2025',
'pred_2026',
'pred_2027',
'pred_2028'
]

In [27]:
y_target = ['real_2018',
'real_2019',
'real_2020',
'real_2021',
'real_2022',
'real_2023',
'pred_2024',
'pred_2025',
'pred_2026',
'pred_2027',
'pred_2028']

In [11]:
#test
df_new = df.loc[df["jenis_simpanan"]=="Tabungan"].copy()
df_new.head()

,kab,prov,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,...,real_2019,real_2020,real_2021,real_2022,real_2023,pred_2024,pred_2025,pred_2026,pred_2027,pred_2028
bps_kab,,,,,,,,,,,,,,,,,,,,,
6171,KOTA PONTIANAK,KALIMANTAN BARAT,676096,206292,115.893439,108.0,5833.772845,59073,55372,52263,...,14176141.56,15636562.28,17466906.94,19076608.21,19685780.82,21337157.77,23183685.09,24793584.13,25402713.87,27054071.57
6172,KOTA SINGKAWANG,KALIMANTAN BARAT,242146,74267,470.199797,452.0,514.985335,21379,20683,20221,...,2940837.55,3441112.76,3814378.76,3926997.56,3887159.25,4540767.94,4914045.53,4949971.63,5063504.80,5640435.20
6101,SAMBAS,KALIMANTAN BARAT,641137,196171,5975.718992,3490.0,107.290353,57514,46696,46150,...,2377673.19,2704488.75,3027703.17,3233951.52,3271162.96,3541798.12,3864977.94,4071264.28,4108450.66,4379103.82
6102,BENGKAYANG,KALIMANTAN BARAT,291752,84997,2017.187608,1785.0,144.633052,28294,24099,24275,...,811342.39,914168.40,1057033.76,1006842.60,903036.69,1101427.67,1244314.24,1146305.94,1138121.15,1288708.08
6103,LANDAK,KALIMANTAN BARAT,406945,117908,12831.320210,6338.0,31.714975,39104,33519,33011,...,674397.61,796223.49,962618.74,871409.19,823819.65,987529.02,1153909.93,1041731.70,1036107.64,1178850.14


In [17]:
#test
df_new = df.loc[df['jenis_simpanan']=="Tabungan"].copy()
x = df_new.drop([
                'kab',
                'prov',
                'jenis_kpi',
                'jenis_simpanan',
                'real_2018',
                'real_2019',
                'real_2020',
                'real_2021',
                'real_2022',
                'real_2023',
                'pred_2024',
                'pred_2025',
                'pred_2026',
                'pred_2027',
                'pred_2028'], axis = 1).copy()
y = df_new['real_2018']

X_train, X_test, y_train, y_test = holdout(x, y, test_size=0.2, random_state=22)


In [18]:

svr = SVR()
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()
mlp = MLPRegressor(hidden_layer_sizes=(100,50,30,15,10, 10))
lr = LinearRegression()
knn = KNeighborsRegressor()


#train
svr.fit(X_train, y_train)
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)
mlp.fit(X_train, y_train)
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


KNeighborsRegressor()

In [19]:
import numpy as np

y_pred_svr = svr.predict(X_test)
y_pred_dt = dt.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_mlp = mlp.predict(X_test)
y_pred_lr = lr.predict(X_test)
y_pred_knn = knn.predict(X_test)

print("SVR")
print("MSE: ",mean_squared_error(y_test,y_pred_svr))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_svr)))
print("R2: ",r2_score(y_test,y_pred_svr))
print("")

print("Decission Tree Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_dt))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_dt)))
print("R2: ",r2_score(y_test,y_pred_dt))
print("")

print("Random Forest Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_rf))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_rf)))
print("R2: ",r2_score(y_test,y_pred_rf))
print("")

print("MLP Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_mlp))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_mlp)))
print("R2: ",r2_score(y_test,y_pred_mlp))
print("")

print("Linear Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_lr))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_lr)))
print("R2: ",r2_score(y_test,y_pred_lr))
print("")

print("KNN")
print("MSE: ",mean_squared_error(y_test,y_pred_knn))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_knn)))
print("R2: ",r2_score(y_test,y_pred_knn))
print("")

#LAG 1
# SVR
# MSE:  46354.59285912892
# RMSE:  215.30116780716477
# R2:  -6.705221132195093

# Decission Tree Regression
# MSE:  6037.452380952381
# RMSE:  77.70104491544744
# R2:  -0.0035662660593165363

# Random Forest Regression
# MSE:  6794.314012950828
# RMSE:  82.42762894170151
# R2:  -0.12937443050042874

# MLP Regression
# MSE:  4084.473084060486
# RMSE:  63.90988252266222
# R2:  0.3210647234713663

# Linear Regression
# MSE:  2394.832091799372
# RMSE:  48.93702168909927
# R2:  0.6019227070364079

# KNN
# MSE:  8284.283809523808
# RMSE:  91.01804112110855
# R2:  -0.3770423756763426

#LAG3
# SVR
# MSE:  43852.88402276194
# RMSE:  209.41080206799728
# R2:  -6.052630152114012

# Decission Tree Regression
# MSE:  4160.475
# RMSE:  64.50174416246432
# R2:  0.33089254935008694

# Random Forest Regression
# MSE:  3547.062980000001
# RMSE:  59.557224414843255
# R2:  0.42954439869426353

# MLP Regression
# MSE:  10022.044968423888
# RMSE:  100.11016416140716
# R2:  -0.6117931147575446

# Linear Regression
# MSE:  2102.168584776421
# RMSE:  45.8494120439556
# R2:  0.6619192129273492

# KNN
# MSE:  5969.379999999999
# RMSE:  77.26176285848
# R2:  0.03997581195402522

#REAL
# SVR
# MSE:  222978500152007.97
# RMSE:  14932464.63756094
# R2:  -0.09595512013717866

# Decission Tree Regression
# MSE:  165246308545011.62
# RMSE:  12854816.550422322
# R2:  0.18780269034811414

# Random Forest Regression
# MSE:  80419592683423.53
# RMSE:  8967697.178396666
# R2:  0.60473200644609

# MLP Regression
# MSE:  90245988161989.62
# RMSE:  9499788.848284451
# R2:  0.556434576739254

# Linear Regression
# MSE:  114980233776330.98
# RMSE:  10722883.650228187
# R2:  0.434864007804185

# KNN
# MSE:  113364343735800.47
# RMSE:  10647269.308879176
# R2:  0.4428062217947303

SVR
MSE:  11970982639920.945
RMSE:  3459910.7849655524
R2:  -0.0926219423256529

Decission Tree Regression
MSE:  7468857493027.213
RMSE:  2732921.055030169
R2:  0.31829843660697255

Random Forest Regression
MSE:  2257671773571.34
RMSE:  1502555.0817095991
R2:  0.793936572078296

MLP Regression
MSE:  2866857819907.372
RMSE:  1693179.7955053009
R2:  0.7383346168164382

Linear Regression
MSE:  2642618959607.115
RMSE:  1625613.4102569143
R2:  0.7588014662352238

KNN
MSE:  6527300888018.437
RMSE:  2554858.291181418
R2:  0.40423669560531916



In [20]:
import pickle
with open("model/my_model_mlp_Tabungan_Nominal_real_2018.pkl", "wb") as f:
  pickle.dump(mlp, f)

with open("model/my_model_rf_Tabungan_Nominal_real_2018.pkl", "wb") as f:
  pickle.dump(rf, f)


In [22]:
for xx in df['jenis_simpanan'].unique():
    df_new = df.loc[df["jenis_simpanan"] == xx].copy()
    for yy in df['jenis_kpi'].unique():
        df_new2 = df_new.loc[df_new["jenis_kpi"] == yy].copy()
        for zz in y_target:
            x = df_new2.drop([
                'kab',
                'prov',
                'jenis_kpi',
                'jenis_simpanan',
                'real_2018',
                'real_2019',
                'real_2020',
                'real_2021',
                'real_2022',
                'real_2023',
                'pred_2024',
                'pred_2025',
                'pred_2026',
                'pred_2027',
                'pred_2028'], axis = 1).copy()
            y = df_new2[zz] 
            X_train, X_test, y_train, y_test = holdout(x, y, test_size=0.2, random_state=22)
            mlp = MLPRegressor(hidden_layer_sizes=(100,50,30,15,10, 10))

            mlp.fit(X_train, y_train)
            y_pred_mlp = mlp.predict(X_test)
            print(xx+"_"+yy+"_"+zz)
            print("MLP Regression")
            print("Target: ", xx)
            print("MSE: ",mean_squared_error(y_test,y_pred_mlp))
            print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_mlp)))
            print("R2: ",r2_score(y_test,y_pred_mlp))
            print("")
            with open("model/my_model_mlp_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(mlp, f)
            
            rf = RandomForestRegressor(n_estimators=175, max_depth=7)

            rf.fit(X_train, y_train)
            y_pred_rf = rf.predict(X_test)
            print(xx+"_"+yy+"_"+zz)
            print("RF Regression")
            print("Target: ", xx)
            print("MSE: ",mean_squared_error(y_test,y_pred_rf))
            print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_rf)))
            print("R2: ",r2_score(y_test,y_pred_rf))
            print("")
            with open("model/my_model_rf_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(rf, f)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Tabungan_Nominal_real_2018
MLP Regression
Target:  Tabungan
MSE:  3275807101754.1377
RMSE:  1809919.087073822
R2:  0.701008778822654

Tabungan_Nominal_real_2018
RF Regression
Target:  Tabungan
MSE:  2103784800180.6438
RMSE:  1450442.966883098
R2:  0.8079822263760512



/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Tabungan_Nominal_real_2019
MLP Regression
Target:  Tabungan
MSE:  3895161579727.2544
RMSE:  1973616.3709615034
R2:  0.6888736149252028

Tabungan_Nominal_real_2019
RF Regression
Target:  Tabungan
MSE:  1400865358599.4783
RMSE:  1183581.5808804554
R2:  0.8881057522065403



/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Giro_Nominal_pred_2026
MLP Regression
Target:  Giro
MSE:  19263808585851.02
RMSE:  4389055.545997455
R2:  0.14426203146730698

Giro_Nominal_pred_2026
RF Regression
Target:  Giro
MSE:  11306073796955.041
RMSE:  3362450.5642395755
R2:  0.49776096559674143



/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Giro_Nominal_pred_2027
MLP Regression
Target:  Giro
MSE:  19424335229356.434
RMSE:  4407304.7579395315
R2:  0.132278761890404

Giro_Nominal_pred_2027
RF Regression
Target:  Giro
MSE:  9731777300261.404
RMSE:  3119579.667240669
R2:  0.5652633797615184



/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Giro_Nominal_pred_2028
MLP Regression
Target:  Giro
MSE:  12739092429332.672
RMSE:  3569186.5220709145
R2:  0.4508653307196946

Giro_Nominal_pred_2028
RF Regression
Target:  Giro
MSE:  11351285483457.088
RMSE:  3369166.8826962383
R2:  0.5106885019915763



In [2]:
#Get data kec
client = bigquery.Client()
query = """
SELECT * FROM `ai4indonesia.geoplan_sbx.ai4_reff_kec`
"""
query_job = client.query(query)
results = query_job.result()

df_kec = results.to_dataframe()

df_kec.head()

,bps_kec_id,kab_name,bps_kab_id,prov_name,kec_name,kmdgr_kec_id,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,...,popses_e,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income
0,1605040,MUSI RAWAS,1605,SUMATERA SELATAN,TUGUMULYO,16.05.01,49017,16172,58.200867,57,...,162,760,1391,5285,5588,1890,1091,167,4,1149531
1,1605080,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA LAKITAN,16.05.02,43544,13197,1747.876038,622,...,142,619,1135,4315,4562,1539,890,137,4,1149531
2,1605070,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA KELINGI,16.05.03,45222,13928,651.289693,742,...,149,655,1199,4551,4815,1626,938,144,4,1149531
3,1605060,MUSI RAWAS,1605,SUMATERA SELATAN,JAYALOKA,16.05.08,16763,5433,157.124750,156,...,54,256,468,1774,1879,634,364,58,4,1149531
4,1605050,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA BELITI,16.05.09,28212,8759,172.835738,173,...,92,410,755,2862,3026,1025,590,91,4,1149531


In [3]:
# df_kec = df_kec.set_index('bps_kec')
df_kec.fillna(0, inplace=True)
df_kec["id_nama"] = df_kec["bps_kec_id"]+"-"+df_kec["kec_name"]
df_kec.head()

,bps_kec_id,kab_name,bps_kab_id,prov_name,kec_name,kmdgr_kec_id,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,...,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income,id_nama
0,1605040,MUSI RAWAS,1605,SUMATERA SELATAN,TUGUMULYO,16.05.01,49017,16172,58.200867,57,...,760,1391,5285,5588,1890,1091,167,4,1149531,1605040-TUGUMULYO
1,1605080,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA LAKITAN,16.05.02,43544,13197,1747.876038,622,...,619,1135,4315,4562,1539,890,137,4,1149531,1605080-MUARA LAKITAN
2,1605070,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA KELINGI,16.05.03,45222,13928,651.289693,742,...,655,1199,4551,4815,1626,938,144,4,1149531,1605070-MUARA KELINGI
3,1605060,MUSI RAWAS,1605,SUMATERA SELATAN,JAYALOKA,16.05.08,16763,5433,157.124750,156,...,256,468,1774,1879,634,364,58,4,1149531,1605060-JAYALOKA
4,1605050,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA BELITI,16.05.09,28212,8759,172.835738,173,...,410,755,2862,3026,1025,590,91,4,1149531,1605050-MUARA BELITI


In [17]:
# df_twr = df_kec.loc[df_kec["id_nama"] == "6101010-SELAKAU"]
df_twr = df_kec.copy()
x = df_twr.drop([
  'kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
x1 = df_twr.drop([
  'kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()

In [18]:
x1.head()


,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,popses_e,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income
0,49017,16172,58.200867,57,842.203943,3971,3608,3381,3472,3911,...,162,760,1391,5285,5588,1890,1091,167,4,1149531
1,43544,13197,1747.876038,622,24.912522,3659,3186,3346,3729,3512,...,142,619,1135,4315,4562,1539,890,137,4,1149531
2,45222,13928,651.289693,742,69.434540,3851,3368,3318,3903,3748,...,149,655,1199,4551,4815,1626,938,144,4,1149531
3,16763,5433,157.124750,156,106.685929,1375,1092,1289,1420,1456,...,54,256,468,1774,1879,634,364,58,4,1149531
4,28212,8759,172.835738,173,163.230130,2383,2186,2296,2455,2305,...,92,410,755,2862,3026,1025,590,91,4,1149531


In [19]:
import pickle
with open("model/simpanan/my_model_mlp_Tabungan_Nominal_real_2018.pkl", "rb") as f:
    mlp = pickle.load(f)

with open("model/simpanan/my_model_rf_Tabungan_Nominal_real_2018.pkl", "rb") as f:
     rf = pickle.load(f)

In [23]:
import numpy as np

y_pred_mlp = mlp.predict(x)
y_pred_rf = rf.predict(x)
# if y_pred_mlp<0:
#     y_pred = y_pred_rf
# else:
#     y_pred = y_pred_mlp
    
# x1['id_nama'] = "6101010-SELAKAU"
x1['jenis_kpi'] = "Nominal"
x1['jenis_simpanan'] = "Tabungan"
x1['pred_year'] = "pred_2018"
x1['value_mlp'] = y_pred_mlp
x1['value_rf'] = y_pred_rf
x1['id_nama'] = df_kec['id_nama']
x1['value'] = np.where(x1['value_mlp'] < 0, x1['value_rf'], x1['value_mlp'])
# x1.head()

df_result = x1.copy()

In [24]:
df_result.head()

,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,jmlh_kk_e,mostses,avrg_income,jenis_kpi,jenis_simpanan,pred_year,value_mlp,value_rf,id_nama,value
0,49017,16172,58.200867,57,842.203943,3971,3608,3381,3472,3911,...,167,4,1149531,Nominal,Tabungan,pred_2018,266271.339343,1.650031e+06,1605040-TUGUMULYO,266271.339343
1,43544,13197,1747.876038,622,24.912522,3659,3186,3346,3729,3512,...,137,4,1149531,Nominal,Tabungan,pred_2018,238094.479651,3.564684e+05,1605080-MUARA LAKITAN,238094.479651
2,45222,13928,651.289693,742,69.434540,3851,3368,3318,3903,3748,...,144,4,1149531,Nominal,Tabungan,pred_2018,240357.710731,7.432116e+05,1605070-MUARA KELINGI,240357.710731
3,16763,5433,157.124750,156,106.685929,1375,1092,1289,1420,1456,...,58,4,1149531,Nominal,Tabungan,pred_2018,257021.359060,1.517231e+06,1605060-JAYALOKA,257021.359060
4,28212,8759,172.835738,173,163.230130,2383,2186,2296,2455,2305,...,91,4,1149531,Nominal,Tabungan,pred_2018,259123.028076,1.550714e+06,1605050-MUARA BELITI,259123.028076


In [25]:
empty_df = pd.DataFrame(columns=df_result.columns)

# Assign the empty DataFrame to the existing DataFrame (effectively deleting values)
df_result = empty_df.copy()

df_result.head()

,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,jmlh_kk_e,mostses,avrg_income,jenis_kpi,jenis_simpanan,pred_year,value_mlp,value_rf,id_nama,value


In [31]:
i=0
x = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
x1 = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
    
for xx in df['jenis_simpanan'].unique():
    x = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
    x1 = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
 
    for yy in df['jenis_kpi'].unique():
        for zz in y_target:
            print(xx+" "+yy+"_"+zz)
            with open("model/simpanan/my_model_mlp_"+xx+"_"+yy+"_"+zz+".pkl", "rb") as f:
                mlp = pickle.load(f)
            with open("model/simpanan/my_model_rf_"+xx+"_"+yy+"_"+zz+".pkl", "rb") as f:
                rf = pickle.load(f)

            y_pred_mlp = mlp.predict(x)
            y_pred_rf = rf.predict(x)
           
            x1['jenis_simpanan'] = xx
            x1['jenis_kpi'] = yy
            x1['pred_year'] = zz 
            x1['value_mlp'] = y_pred_mlp
            x1['value_rf'] = y_pred_rf
            x1['id_nama'] = df_kec['id_nama']
            x1['value'] = np.where(x1['value_mlp'] < 0, x1['value_rf'], x1['value_mlp'])    
            
            # Concatenate DataFrames
            df_result= pd.concat([df_result, x1], ignore_index=True)
            # df_result = df_result.append(x1, ignore_index=True)
            print(str(i)+"_pred_done")
            i=i+1

Tabungan Nominal_real_2018
0_pred_done
Tabungan Nominal_real_2019
1_pred_done
Tabungan Nominal_real_2020
2_pred_done
Tabungan Nominal_real_2021
3_pred_done
Tabungan Nominal_real_2022
4_pred_done
Tabungan Nominal_real_2023
5_pred_done
Tabungan Nominal_pred_2024
6_pred_done
Tabungan Nominal_pred_2025
7_pred_done
Tabungan Nominal_pred_2026
8_pred_done
Tabungan Nominal_pred_2027
9_pred_done
Tabungan Nominal_pred_2028
10_pred_done
Simpanan Berjangka Nominal_real_2018
11_pred_done
Simpanan Berjangka Nominal_real_2019
12_pred_done
Simpanan Berjangka Nominal_real_2020
13_pred_done
Simpanan Berjangka Nominal_real_2021
14_pred_done
Simpanan Berjangka Nominal_real_2022
15_pred_done
Simpanan Berjangka Nominal_real_2023
16_pred_done
Simpanan Berjangka Nominal_pred_2024
17_pred_done
Simpanan Berjangka Nominal_pred_2025
18_pred_done
Simpanan Berjangka Nominal_pred_2026
19_pred_done
Simpanan Berjangka Nominal_pred_2027
20_pred_done
Simpanan Berjangka Nominal_pred_2028
21_pred_done
Giro Nominal_real_20

In [32]:
df_result

,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,jmlh_kk_e,mostses,avrg_income,jenis_kpi,jenis_simpanan,pred_year,value_mlp,value_rf,id_nama,value
0,49017,16172,58.200867,57,842.203943,3971,3608,3381,3472,3911,...,167,4,1149531,Nominal,Tabungan,real_2018,266271.339343,1.650031e+06,1605040-TUGUMULYO,266271.339343
1,43544,13197,1747.876038,622,24.912522,3659,3186,3346,3729,3512,...,137,4,1149531,Nominal,Tabungan,real_2018,238094.479651,3.564684e+05,1605080-MUARA LAKITAN,238094.479651
2,45222,13928,651.289693,742,69.434540,3851,3368,3318,3903,3748,...,144,4,1149531,Nominal,Tabungan,real_2018,240357.710731,7.432116e+05,1605070-MUARA KELINGI,240357.710731
3,16763,5433,157.124750,156,106.685929,1375,1092,1289,1420,1456,...,58,4,1149531,Nominal,Tabungan,real_2018,257021.359060,1.517231e+06,1605060-JAYALOKA,257021.359060
4,28212,8759,172.835738,173,163.230130,2383,2186,2296,2455,2305,...,91,4,1149531,Nominal,Tabungan,real_2018,259123.028076,1.550714e+06,1605050-MUARA BELITI,259123.028076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247039,2565,743,51.330128,38,49.970652,339,322,279,237,165,...,0,7,1160643,Nominal,Giro,pred_2028,237799.152008,1.878229e+06,9110073-AYAMARU TENGAH,237799.152008
247040,1539,500,74.917416,57,20.542620,202,157,136,117,98,...,0,7,1160643,Nominal,Giro,pred_2028,233090.695967,1.878229e+06,9110074-AYAMARU BARAT,233090.695967
247041,1462,488,123.924255,91,11.797529,155,148,125,121,82,...,0,7,1160643,Nominal,Giro,pred_2028,232582.358302,1.878229e+06,9110021-AITINYO TENGAH,232582.358302
247042,1534,539,29.401490,25,52.174226,155,129,142,125,67,...,0,7,1160643,Nominal,Giro,pred_2028,233310.091355,1.878229e+06,9110051-AITINYO RAYA,233310.091355


In [128]:
df_result.to_excel("Hasil_Simpanan_Kecamatan.xlsx",index=False)

In [35]:
import pandas as pd
from google.cloud import bigquery

table_id = 'ai4indonesia.geoplan_sbx.ai4_data_simpanan_kec_result'

# Write DataFrame to BigQuery
df_result.to_gbq(destination_table=table_id, project_id='ai4indonesia')

100%|██████████| 1/1 [00:00<00:00, 10699.76it/s]
